# Analyze cell state dynamics dependent on colocalization
Exdyn can estimate cell state dynamics dependent on multivariate extrinsic factors. Here, we will go through analysis of cell state dynamics of human skin squamous cell carcinoma dependent on neighboring cell colocalization strength.

## Model set up and optimzaiton
Exdyn accepts AnnData object for dynamics estimation. The model parameters such as the number of neural network layers can be specified through a dictionary of the parameters.

In [2]:
import scanpy as sc
from src import workflow, visualization

adata_file_path = 'data/scc/adata.h5ad'
coloc_adata_file_path = 'data/scc/adata.h5ad'

adata = sc.read_h5ad(adata_file_path)
model_params = {
            'x_dim': 100,
            'z_dim': 10,
            'enc_z_h_dim': 128, 'enc_d_h_dim': 128, 'dec_z_h_dim': 128,
            'num_enc_z_layers': 2, 'num_enc_d_layers': 2,
            'num_dec_z_layers': 2, 'use_ambient': False, 'use_vamp': True, 'no_d_kld': False, 'decreasing_temp': False, 'dec_temp_steps': 30000, 'loss_mode': 'nb',
            'dyn_loss_weight': 1
}
checkpoint_dirname = f'{out_dir}/checkpoint'
importlib.reload(workflow)
importlib.reload(commons)
importlib.reload(modules)
adata.obs['sample'] = 'Sample'
est_adata, lit_envdyn = workflow.conduct_cvicdyf_inference(adata, model_params, checkpoint_dirname, batch_size=128, two_step=False, dyn_mode=False, epoch=1000, patience=50, module=modules.Cvicdyf, use_highly_variable=False, batch_key='sample', condition_key='coloc', cond_in_obsm=True)


FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'data/scc/adata.h5ad', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

## Visualize cell state dynamics dependent on colocalization
Firstly, we visulize cell state dynamics with colocalization profiles originally assgned to each cell.

In [ ]:
visualization.plot_mean_flow(est_adata, cluster_key='Annotation', legend_loc='on data').

## Principal mode analysis of cell state dynamics perturbation by extrinsic factors
Next, we conduct singular value decompositon of Jacobian matrix of cell state dynamics differentiated by extrinsic factors. This enable us to analyze each mode of the principan perturbation. 

In [ ]:
est_adata = workflow.conduct_jacobian_analysis(est_adata, model_params, top_modes=5)

We can visualize the effect of each perturbation on cell state dynamics.

In [ ]:
fig, axes = plt.subplots(1, display_modes, figsize=(6 * display_modes, 5 * 1))
for mode in range(display_modes):
   ax = axes[mode] 
   visualization.diff_visualize(est_adata.obsm[f'dd_dc_embed{mode}'], est_adata.obsm['X_umap'], ax)
   ax.set_title(f"Dynamics difference of mode {mode + 1}")

fig.show()

We can also analyze the contributions of each feature of extrinsic factors to each principal mode, which are stored in `adata.uns['dd_dc_v']`.

In [ ]:
adata.uns['dd_dc_v']

## Identify colocalized cells contributing to the induction of specific cell states
We will show how to identify colocalized cells which may induce TSK by cell cell interaction. Firstly, we can identify cancer cells which are directed toward TSK by positive mode 1 perturbation.

In [ ]:
mode = 1
cluster = 'TSK'
sign = 1
est_adata = workflow.analyze_flux_into_cluster(est_adata, cluster, mode, 'level2_celltype')
sc.pl.umap(est_adata, color=f'mode{mode}_flux_into_{cluster}')

Finally, we can estimate which genes are affected by the perturbation in the cells directed by the mode

In [ ]:

est_adata = workflow.calc_covdiff(est_adata, model, mode, sign, condition_key='coloc', batch_key='sample', sep_num=5)